In [64]:
class ParseError(Exception):
    pass

def lookahead(tokens):
    if not tokens:
        raise ParseError("no tokens")
    return tokens[0]

def match_tok(tokens, expected):
    if not tokens:
        raise ParseError("bad match")
    if tokens[0] == expected:
        return tokens[1:]
    else:
        raise ParseError("bad match")

def parse_T(tokens):
    """
    Parse T → w T | ε
    Matches zero or more 'w' tokens.
    """
    if tokens and lookahead(tokens) == "w":
        # Match 'w' and recurse
        return parse_T(match_tok(tokens, "w"))
    else:
        # ε case: do nothing, return tokens
        return tokens

def parse_S(tokens):
    """
    Parse S → x y S z | x T y z
    Uses LL(2) lookahead to decide.
    """
    tokens_after_x = match_tok(tokens, "x")  # Match 'x'
    next_tok = lookahead(tokens_after_x)    # Look ahead at 1 token

    if next_tok == "y":
        # Look ahead at the second token
        tokens_after_y = match_tok(tokens_after_x, "y")
        next_next_tok = lookahead(tokens_after_y)

        if next_next_tok == "x":
            # xySz branch
            tokens_after_subS = parse_S(tokens_after_y)
            tokens_after_z = match_tok(tokens_after_subS, "z")
            return tokens_after_z
        else:
            # xTyz branch
            tokens_after_T = parse_T(tokens_after_x)
            tokens_after_y = match_tok(tokens_after_T, "y")
            tokens_after_z = match_tok(tokens_after_y, "z")
            return tokens_after_z

    elif next_tok == "w":
        # Definitely the xTyz branch
        tokens_after_T = parse_T(tokens_after_x)
        tokens_after_y = match_tok(tokens_after_T, "y")
        tokens_after_z = match_tok(tokens_after_y, "z")
        return tokens_after_z

    else:
        raise ParseError(f"Expected 'y' or 'w' after 'x', got '{next_tok}'")

def parse(tokens):
    remaining = parse_S(tokens)
    if remaining:
        raise ParseError(f"Leftover tokens after parsing S: {remaining}")
    return "Parse succeeded!"


In [67]:
# Example input tokens
tokens = ["x", "y", "x", "y", "z", "z"]

# Try parsing and handle any errors
try:
    result = parse(tokens)
    print(result)  # Should print "Parse succeeded!" if the input is valid
except ParseError as e:
    print(f"Parse failed: {e}")


Parse succeeded!


In [66]:
# Define test cases
test_cases = [
    # Valid cases
    (["x", "y", "z"], "Parse succeeded!"),  # S → x T y z (T → ε)
    (["x", "w", "y", "z"], "Parse succeeded!"),  # S → x T y z (T → w T)
    (["x", "y", "x", "y", "z", "z"], "Parse succeeded!"),  # S → x y S z
    (["x", "w", "w", "y", "z"], "Parse succeeded!"),  # S → x T y z (T → ww)
    
    # Invalid cases
    (["x", "z"], "Parse failed: Expected 'y' or 'w' after 'x', got 'z'"),  # Missing 'y'
    (["y", "x", "y", "z"], "Parse failed: bad match"),  # Missing 'x' at start
    (["x", "y", "z", "x"], "Parse failed: Leftover tokens after parsing S: ['x']"),  # Extra token
    ([], "Parse failed: bad match"),  # Empty input
]

# Run the test cases
for i, (tokens, expected) in enumerate(test_cases):
    try:
        result = parse(tokens)
        print(f"Test case {i + 1}: Passed (Result: {result})")
    except ParseError as e:
        result = f"Parse failed: {e}"
        if result == expected:
            print(f"Test case {i + 1}: Passed (Result: {result})")
        else:
            print(f"Test case {i + 1}: Failed (Expected: {expected}, Got: {result})")


Test case 1: Passed (Result: Parse succeeded!)
Test case 2: Passed (Result: Parse succeeded!)
Test case 3: Passed (Result: Parse succeeded!)
Test case 4: Passed (Result: Parse succeeded!)
Test case 5: Passed (Result: Parse failed: Expected 'y' or 'w' after 'x', got 'z')
Test case 6: Passed (Result: Parse failed: bad match)
Test case 7: Passed (Result: Parse failed: Leftover tokens after parsing S: ['x'])
Test case 8: Passed (Result: Parse failed: bad match)
